1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [59]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, classification_report, precision_recall_curve, confusion_matrix

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler



import matplotlib.pyplot as plt

%matplotlib inline

In [60]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [61]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [62]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [63]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [64]:
gender = Pipeline([
                ('selector', FeatureSelector(column='Gender')),
                ('ohe', OHEEncoder(key='Gender'))
            ])
gender.fit(X_train)
gender.transform(X_test).head(3)

,Gender_Female,Gender_Male
9394,1,0
898,1,0
2398,1,0


In [65]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [66]:
feats = FeatureUnion(final_transformers)
feats.fit(X_train)

FeatureUnion(transformer_list=[('Geography',
                                Pipeline(steps=[('selector',
                                                 FeatureSelector(column='Geography')),
                                                ('ohe',
                                                 OHEEncoder(key='Geography'))])),
                               ('Gender',
                                Pipeline(steps=[('selector',
                                                 FeatureSelector(column='Gender')),
                                                ('ohe',
                                                 OHEEncoder(key='Gender'))])),
                               ('Tenure',
                                Pipeline(steps=[('selector',
                                                 FeatureSelector(column='Tenure')),
                                                ('ohe',
                                                 OHEEncoder(key=...
                                    

In [67]:
X_train_tr = feats.transform(X_train)
X_test_tr = feats.transform(X_test)

In [71]:
model_list = [GradientBoostingClassifier(), LogisticRegression(), RandomForestClassifier()]
res = pd.DataFrame(columns=['model', 'recall', 'precision', 'f-score'])
for model in model_list:
    model.fit(X_train_tr, y_train)
    pred_train = model.predict(X_train_tr)
    pred_test = model.predict(X_test_tr)
    pr = precision_score(y_test, pred_test)
    re = recall_score(y_test, pred_test)
    f1 = 2 * (pr * re) / (pr + re)
    res = res.append({
        'model': f'{model.__class__.__name__}',
        'recall': re,
        'precision': pr,
        'f-score': f1
    }, ignore_index=True)
res

,model,recall,precision,f-score
0,GradientBoostingClassifier,0.518664,0.745763,0.611819
1,LogisticRegression,0.235756,0.574163,0.334262
2,RandomForestClassifier,0.485265,0.760000,0.592326


Результаты показывают, что лучше справляется градиентный бустинг. По мнению этой модели мы охватим наибольшее число клиентов для удержания и при этом удержим того, кого действительно надо удержать.